In [1]:
import os
import kagglehub

# Set Kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

# Verify Kaggle setup
kagglehub.login() # This will prompt you for your credentials.

Kaggle API authenticated successfully!


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
import kagglehub

# Download latest version
path = kagglehub.model_download("keras/paligemma/keras/pali_gemma_3b_224")

print("Path to model files:", path)

Download already complete (4543 bytes).
Download already complete (693 bytes).
Download already complete (184 bytes).
Download already complete (2376 bytes).
Download already complete (614 bytes).
Download already complete (1092 bytes).
Download already complete (4264023 bytes).
Resuming download from 4185915392 bytes (1662057920 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/models/keras/paligemma/keras/pali_gemma_3b_224/4/download/model.weights.h5 (4185915392/5847973312) bytes left.


100%|██████████| 5.45G/5.45G [00:24<00:00, 67.0MB/s]

Path to model files: C:\Users\rbrul\.cache\kagglehub\models\keras\paligemma\keras\pali_gemma_3b_224\4


In [7]:
import keras
import keras_nlp
from IPython.display import HTML, display
from PIL import Image
import io
import requests
from transformers import AutoProcessor, AutoTokenizer, PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torch
os.environ["KERAS_BACKEND"] = "jax"
keras.config.set_floatx("bfloat16")


RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace prims; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:488; latest registration was registered at /dev/null:488

In [14]:
model_id = "google/paligemma-3b-mix-224"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map=device,
    revision="bfloat16",
).eval()
processor = AutoProcessor.from_pretrained(model_id)

# Instruct the model to create a caption in Spanish
prompt = "caption es"
model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)
input_len = model_inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]
    decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)



ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`